<a href="https://colab.research.google.com/github/NuzhatR10/C201210_CSE-3532_E-Commerce-Website/blob/main/sentiment_analysis_from_twitter_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle




  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
! mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 77% 62.0M/80.9M [00:00<00:00, 210MB/s]
100% 80.9M/80.9M [00:00<00:00, 211MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
twitter_data.replace({'target': {4: 1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0==negative tweets
1==positive tweets


stemming


In [ ]:
def preprocess_text(content):
    content = re.sub('[^a-zA-Z]', ' ', content)
    content = content.lower()
    return content

In [ ]:
twitter_data['processed_text'] = twitter_data['text'].apply(preprocess_text)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,processed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com y zl awww t...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dived many times for the ball man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no it s not behaving at all ...


In [ ]:
X = twitter_data['processed_text'].values

In [ ]:
Y = twitter_data['target'].values

separeting data and level


In [ ]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [ ]:
print(X)

[' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d'
 'is upset that he can t update his facebook by texting it    and might cry as a result  school today also  blah '
 ' kenichan i dived many times for the ball  managed to save      the rest go out of bounds'
 ... 'are you ready for your mojo makeover  ask me for details '
 'happy   th birthday to my boo of alll time    tupac amaru shakur '
 'happy  charitytuesday  thenspcc  sparkscharity  speakinguph h ']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


splitting dataset


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
max_words = 5000  # Adjust as needed
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100  # Adjust as needed
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
embedding_dim = 50  # Adjust as needed
model = Sequential([ Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 50)           250000    
                                                                 
 conv1d_1 (Conv1D)           (None, 96, 128)           32128     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 282257 (1.08 MB)
Trainable params: 282257 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
epochs = 5  # Adjust as needed
model.fit(X_train_padded, Y_train, epochs=epochs, validation_data=(X_test_padded, Y_test))

Epoch 1/5
40000/40000 [==============================] - 753s 19ms/step - loss: 0.4181 - accuracy: 0.8075 - val_loss: 0.3977 - val_accuracy: 0.8185
Epoch 2/5
40000/40000 [==============================] - 787s 20ms/step - loss: 0.3878 - accuracy: 0.8251 - val_loss: 0.3934 - val_accuracy: 0.8216
Epoch 3/5
40000/40000 [==============================] - 830s 21ms/step - loss: 0.3723 - accuracy: 0.8339 - val_loss: 0.3918 - val_accuracy: 0.8231
Epoch 4/5
40000/40000 [==============================] - 808s 20ms/step - loss: 0.3596 - accuracy: 0.8406 - val_loss: 0.3996 - val_accuracy: 0.8213
Epoch 5/5
40000/40000 [==============================] - 804s 20ms/step - loss: 0.3483 - accuracy: 0.8468 - val_loss: 0.4011 - val_accuracy: 0.8194


In [ ]:
train_loss, train_accuracy = model.evaluate(X_train_padded, Y_train)
print('Accuracy score on the training data:', train_accuracy)

40000/40000 [==============================] - 227s 6ms/step - loss: 0.3200 - accuracy: 0.8627
Accuracy score on the training data: 0.8627179861068726


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, Y_test)
print('Accuracy score on the test data:', test_accuracy)

10000/10000 [==============================] - 54s 5ms/step - loss: 0.4011 - accuracy: 0.8194
Accuracy score on the test data: 0.819434404373169
